In [19]:
# coding: utf-8

import matplotlib.pyplot as plt
import pandas as pd 
import sys 
import matplotlib 
import functools
import numpy as np
import time
import multiprocessing  
import random
#import spectrum
import KSTEST
import itertools
import os.path

from pylab import plot
from scipy import signal
from pandas import DataFrame, read_csv
from scipy import stats
#from spectrum import *
from datetime import timedelta, date, datetime

from multiprocessing import Pool

#Clean Cells That Have only 0 values for 
def CleanNullCells(ZoneDataFrame):
    for kpi in column_kpiselection:
        for cell in ZoneDataFrame['CellName'].unique():
            CelluleData = ZoneDataFrame.loc[ZoneDataFrame['CellName']==cell][['DAY_',kpi]]
            if(CelluleData[kpi] == 0).all():
                  ZoneDataFrame = ZoneDataFrame.loc[ZoneDataFrame['CellName'] != cell]
    return(ZoneDataFrame) 




def KSTestPerDay(cell1,cell2,kpi):
    cell1_kpi = alldata_kpis.loc[alldata_kpis['CellName']==cell1][['DAY_',kpi]]
    cell1_kpi['DAY_'] = pd.to_datetime(cell1_kpi['DAY_'],infer_datetime_format=True)    
    cell2_kpi = alldata_kpis.loc[alldata_kpis['CellName']==cell2][['DAY_',kpi]]
    cell2_kpi['DAY_'] = pd.to_datetime(cell2_kpi['DAY_'],infer_datetime_format=True)
    
    KS = pd.DataFrame()
    for day in np.array([datetime(2018,8,7)+timedelta(days=i) for i in range(0,22)]):
        dayvalue = pd.DataFrame()
        dayvalue['Day'] = [day]
        mask1 = (cell1_kpi['DAY_'] >=day) & (cell1_kpi['DAY_'] <day+timedelta(days=1))
        mask2 = (cell2_kpi['DAY_'] >=day) & (cell2_kpi['DAY_'] <day+timedelta(days=1))

        dayvalue['pvalue'] = [stats.ks_2samp(cell1_kpi.loc[mask1][kpi],cell2_kpi.loc[mask2][kpi])[1]]
        KS = KS.append(dayvalue,ignore_index = True)
    return(KS)

def KSTest(ZoneDataFrame,cell1,cell2,kpiList,dayList, wh = True):
    auxList = [kpi for kpi in kpiList]
    auxList.append('DAY_')
    cell1_kpi = ZoneDataFrame.loc[ZoneDataFrame['CellName']==cell1][auxList]
    cell1_kpi['DAY_'] = pd.to_datetime(cell1_kpi['DAY_'],infer_datetime_format=True)
    cell1_kpi = cell1_kpi.set_index('DAY_')
    cell2_kpi = ZoneDataFrame.loc[ZoneDataFrame['CellName']==cell2][auxList]
    cell2_kpi['DAY_'] = pd.to_datetime(cell2_kpi['DAY_'],infer_datetime_format=True)
    cell2_kpi = cell2_kpi.set_index('DAY_')
    out = {}
    if wh:
        cell1_kpi = cell1_kpi.between_time('7:00','23:00')
        cell1_kpi['DAY_'] = cell1_kpi.index
        cell1_kpi['DAY_'] = pd.to_datetime(cell1_kpi['DAY_'],infer_datetime_format=True)
        cell2_kpi = cell2_kpi.between_time('7:00','23:00')
        cell2_kpi['DAY_'] = cell2_kpi.index
        cell2_kpi['DAY_'] = pd.to_datetime(cell2_kpi['DAY_'],infer_datetime_format=True)
    for day in dayList:
        out[day] = {}
        mask1 = (cell1_kpi['DAY_'] >=day) & (cell1_kpi['DAY_'] <day+timedelta(days=1) ) 
        mask2 = (cell2_kpi['DAY_'] >=day) & (cell2_kpi['DAY_'] <day+timedelta(days=1) )
        for kpi in kpiList:
            pvalue = [stats.ks_2samp(cell1_kpi.loc[mask1][kpi],cell2_kpi.loc[mask2][kpi])[1]]
            out[day][kpi] = pvalue
    return(out)

def KSTestDay(ZoneDataFrame,cell1,cell2,kpi,day):
    cell1_kpi = ZoneDataFrame.loc[ZoneDataFrame['CellName']==cell1][['DAY_',kpi]]
    cell1_kpi['DAY_'] = pd.to_datetime(cell1_kpi['DAY_'],infer_datetime_format=True)
    cell1_kpi.loc[cell1_kpi['DAY_'] ==day]
    cell2_kpi = ZoneDataFrame.loc[ZoneDataFrame['CellName']==cell2][['DAY_',kpi]]
    cell2_kpi['DAY_'] = pd.to_datetime(cell2_kpi['DAY_'],infer_datetime_format=True)
    cell2_kpi.loc[cell2_kpi['DAY_'] ==day]
    return(stats.ks_2samp(cell1_kpi[kpi],cell2_kpi[kpi])[1])



def KSTestPerWeekDay(cell1,cell2,kpi):
    cell1_kpi = alldata_kpis.loc[alldata_kpis['CellName']==cell1][['DAY_',kpi]]   
    cell1_kpi['DAY_'] = pd.to_datetime(cell1_kpi['DAY_'],infer_datetime_format=True)
    cell2_kpi = alldata_kpis.loc[alldata_kpis['CellName']==cell2][['DAY_',kpi]]
    cell2_kpi['DAY_'] = pd.to_datetime(cell2_kpi['DAY_'],infer_datetime_format=True)
  
    KS = pd.DataFrame()
    for day in range(0,7):
        dayvalue = pd.DataFrame()
        dayvalue['Day'] = [day]
        mask1 = (cell1_kpi['DAY_'].dt.dayofweek==day)
        mask2 = (cell2_kpi['DAY_'].dt.dayofweek==day)
        dayvalue['pvalue'] = [stats.ks_2samp(cell1_kpi.loc[mask1][kpi],cell2_kpi.loc[mask2][kpi])[1]]
        KS = KS.append(dayvalue,ignore_index = True)
    return(KS)

def KSTestAllCellsPerDayAverage(ZoneDataFrame,kpi):
    KS = pd.DataFrame()
    alreadyDoneCouples = []
    for cell1 in ZoneDataFrame['CellName'].unique():
        for cell2 in ZoneDataFrame['CellName'].unique():
            if cell1 == cell2:
                KS.at[str(cell1),str(cell2)] = float(1)
            elif [cell2,cell1] in alreadyDoneCouples:
                KS.at[str(cell1),str(cell2)] = KS.loc[str(cell2)][str(cell1)]
            else:
                avg = KSTestPerDay(cell1,cell2,kpi)['pvalue'].mean()
                KS.at[cell1,cell2] = avg
                alreadyDoneCouples.append([cell1,cell2])
    return(KS)

def KSTestAllCellsDay(ZoneDataFrame,kpi,day):
    KS = pd.DataFrame()
    alreadyDoneCouples = []
    for cell1 in ZoneDataFrame['CellName'].unique():
        for cell2 in ZoneDataFrame['CellName'].unique():
            if cell1 == cell2:
                KS.at[str(cell1),str(cell2)] = float(1)
            elif [cell2,cell1] in alreadyDoneCouples:
                KS.at[str(cell1),str(cell2)] = KS.loc[str(cell2)][str(cell1)]
            else:
                avg = KSTestDay(ZoneDataFrame,cell1,cell2,kpi,day)
                KS.at[cell1,cell2] = avg
                alreadyDoneCouples.append([cell1,cell2])
    return(KS)

def KSTestAllCells(ZoneDataFrame,kpiList,dayList):
    out = {}
    for day in dayList:
        out[day]={}
        for kpi in kpiList:
            out[day][kpi] = pd.DataFrame()
    alreadyDoneCouples = []
    for cell1 in ZoneDataFrame['CellName'].unique():
        for cell2 in ZoneDataFrame['CellName'].unique():
            if cell1 == cell2:
                for day in dayList:
                    for kpi in kpiList:
                        out[day][kpi].at[str(cell1),str(cell2)] = float(1)
            elif [cell2,cell1] in alreadyDoneCouples:
                for day in dayList:
                    for kpi in kpiList:
                        out[day][kpi].at[str(cell1),str(cell2)] = out[day][kpi].loc[str(cell2)][str(cell1)]
            else:
                kstest = KSTest(ZoneDataFrame,cell1,cell2,kpiList,dayList)
                for day in kstest:
                    for kpi in kstest[day]:
                        out[day][kpi].at[cell1,cell2] = kstest[day][kpi][0]
                alreadyDoneCouples.append([cell1,cell2])
    return(out)

def KSTestAllCellsPerDayStd(ZoneDataFrame,kpi):
    KS = pd.DataFrame()
    alreadyDoneCouples = []
    for cell1 in ZoneDataFrame['CellName'].unique():
        for cell2 in ZoneDataFrame['CellName'].unique():
            if cell1 == cell2:
                KS.loc[str(cell1),str(cell2)] = float(0)
            elif [cell2,cell1] in alreadyDoneCouples:
                KS.at[str(cell1),str(cell2)] = KS.loc[str(cell2)][str(cell1)]
            else:
                std = KSTestPerDay(cell1,cell2,kpi)['pvalue'].std()
                KS.at[str(cell1),str(cell2)] = float(std)
                alreadyDoneCouples.append([cell1,cell2])
    return(KS)



def KSTestAllCellsPerWeekDayAverage(ZoneDataFrame,kpi):
    KS = pd.DataFrame()
    alreadyDoneCouples = []
    for cell1 in ZoneDataFrame['CellName'].unique():
        for cell2 in ZoneDataFrame['CellName'].unique():
            if cell1 == cell2:
                KS.at[str(cell1),str(cell2)] = float(1)
            elif [cell2,cell1] in alreadyDoneCouples:
                KS.at[str(cell1),str(cell2)] = KS.loc[str(cell2)][str(cell1)]
            else:
                avg = KSTestPerWeekDay(cell1,cell2,kpi)['pvalue'].mean()
                KS.at[cell1,cell2] = avg
                alreadyDoneCouples.append([cell1,cell2])
    return(KS)


def KSTestAllCellsPerWeekDayStd(ZoneDataFrame,kpi):
    KS = pd.DataFrame()
    alreadyDoneCouples = []
    for cell1 in ZoneDataFrame['CellName'].unique():
        for cell2 in ZoneDataFrame['CellName'].unique():
            if cell1 == cell2:
                KS.loc[str(cell1),str(cell2)] = float(0)
            elif [cell2,cell1] in alreadyDoneCouples:
                KS.at[str(cell1),str(cell2)] = KS.loc[str(cell2)][str(cell1)]
            else:
                std = KSTestPerWeekDay(cell1,cell2,kpi)['pvalue'].std()
                KS.at[str(cell1),str(cell2)] = float(std)
                alreadyDoneCouples.append([cell1,cell2])
    return(KS)



def getIdenticallyDistributedUnits(KSTestMatrix, alreadyDeletedCells = []): 
    count = 0
    totalpvalues = float(0)
    ToDeleteCell = 0
    for cell in KSTestMatrix.index.unique():
        A = np.array(KSTestMatrix[cell].values)
        A = A[np.where(A<0.1)]
        if len(A)>0:
            if count < len(A):
                ToDeleteCell = cell 
                count = len(A)
                totalpvalues = sum(A)
            elif (count == len(A)) & (totalpvalues < float(sum(A))):
                ToDeleteCell = cell 
                count = len(A)
                totalpvalues = sum(A)
    if ToDeleteCell == 0:
        return([KSTestMatrix,alreadyDeletedCells])
    else:
        alreadyDeletedCells.append(ToDeleteCell)
        FiltredMatrix = KSTestMatrix.drop([ToDeleteCell], axis = 0)
        FiltredMatrix = FiltredMatrix.drop([ToDeleteCell], axis = 1)
        return(getIdenticallyDistributedUnits(FiltredMatrix,alreadyDeletedCells))
        

def getIdenticallyDistributedClusters(KSTestMatrix, partialResults=[]):
    Cluster = getIdenticallyDistributedUnits(KSTestMatrix,[])[0]
    DeletedCells = getIdenticallyDistributedUnits(KSTestMatrix,[])[1]
    partialResults.append(Cluster)
    if Cluster.size!=KSTestMatrix.size:
        Reste = KSTestMatrix.loc[DeletedCells][DeletedCells]
        return(getIdenticallyDistributedClusters(Reste,partialResults))
    else:
        return(partialResults)


def generateLabeledDataFromClusters(ClusterList,kpi):
    LabeledData = pd.DataFrame()
    Label = 0
    for cluster in ClusterList:
        for clustercell in cluster.index:
            LabeledData.loc[str(clustercell),str(kpi)+' Label'] = Label
        Label = Label + 1
    return(LabeledData)


def generateFeaturesFromCellList(CellList):
    FeaturesData = pd.DataFrame()
    for cell in CellList:
        FeaturesData.loc[cell,'Site'] = int(cell[1:len(cell)-1])
        if cell.startswith('D'):
            FeaturesData.loc[cell,'Tec'] = 0
            if cell[-1] =='A':
                FeaturesData.loc[cell,'Azimuth'] = 0
            elif cell[-1] =='B':
                FeaturesData.loc[cell,'Azimuth'] = 1
            elif cell[-1] =='C':
                FeaturesData.loc[cell,'Azimuth'] = 2    
        elif cell.startswith('L'):
            FeaturesData.loc[cell,'Tec'] = 1
            if cell[-1] =='A':
                FeaturesData.loc[cell,'Azimuth'] = 0
            elif cell[-1] =='B':
                FeaturesData.loc[cell,'Azimuth'] = 1
            elif cell[-1] =='C':
                FeaturesData.loc[cell,'Azimuth'] = 2
        elif cell.startswith('T'):
            FeaturesData.loc[cell,'Tec'] = 2
            if cell[-1] =='A':
                FeaturesData.loc[cell,'Azimuth'] = 0
            elif cell[-1] =='B':
                FeaturesData.loc[cell,'Azimuth'] = 1
            elif cell[-1] =='C':
                FeaturesData.loc[cell,'Azimuth'] = 2
    return(FeaturesData)


def generateKpiLabeledDataFromExcelKSTEST(excelFileLocation, kpiList):
    allKpiDataFrame = pd.DataFrame()
    i = 0
    for kpi in kpiList:
        kpiKSTEST = pd.read_excel(excelFileLocation, sheet_name = kpi)
        kpiClusters = getIdenticallyDistributedClusters(kpiKSTEST,[])
        kpiLabeledData = generateLabeledDataFromClusters(kpiClusters,kpi)
        if i == 0:
            cellFeatures = generateFeaturesFromCellList(kpiKSTEST.index)
            allKpiDataFrame = cellFeatures.join(kpiLabeledData)
            i = i+1
        else:
            allKpiDataFrame = allKpiDataFrame.join(kpiLabeledData) 
    return(allKpiDataFrame)

def generateKpiLabeledDataFromExcelKSTEST(excelFileLocation, kpiList):
    allKpiDataFrame = pd.DataFrame()
    time_struct={}
    i = 0
    for kpi in kpiList:
        kpiKSTEST = pd.read_excel(excelFileLocation, sheet_name = kpi)
        time_struct[kpi]={}
        time_struct[kpi]["delta_t"]=[]
        start_time = time.time()
        kpiClusters = getIdenticallyDistributedClusters(kpiKSTEST,[])
        end_time = time.time()
        time_struct[kpi]["delta_t"].append(end_time-start_time)
 
        kpiLabeledData = generateLabeledDataFromClusters(kpiClusters,kpi)
        if i == 0:
            cellFeatures = generateFeaturesFromCellList(kpiKSTEST.index)
            allKpiDataFrame = cellFeatures.join(kpiLabeledData)
            i = i+1
        else:
            allKpiDataFrame = allKpiDataFrame.join(kpiLabeledData) 
    return(allKpiDataFrame,time_struct)

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

def KSTestAllCellsPerDayAverageParallel(ZoneDataFrame,kpi):
    C = list(itertools.combinations(ZoneDataFrame['CellName'].unique(),2))
    A = [(ZoneDataFrame, c[0],c[1],kpi) for c in C]
    if __name__ == '__main__':
        from datetime import timedelta, date, datetime
        pool = Pool()
        t = time.time()
        print(time.time()-t)
        results = pool.starmap(KSTEST.KSTestPerDayAverageParallelFunc, A)
    del A
    del C
    return(results)

def KSTestAllCellsParallel(ZoneDataFrame,kpiList,dayList):
    C = list(itertools.combinations(ZoneDataFrame['CellName'].unique(),2))
    A = [(ZoneDataFrame, c[0],c[1],kpiList,dayList) for c in C]
    if __name__ == '__main__':
        from datetime import timedelta, date, datetime
        pool = Pool()
        t = time.time()
        print(time.time()-t)
        results = pool.starmap(KSTEST.KSTestParallelFunc, A)
    del A
    out = {}
    for day in dayList:
        out[day]={}
        for kpi in kpiList:
            out[day][kpi] = pd.DataFrame()
            
    i = 0        
    for c in C:
        r = results[i]
        i = i + 1
        for day in dayList:
                    for kpi in kpiList:
                        out[day][kpi].at[str(c[0]),str(c[0])] = float(1)
        for day in kstest:
                    for kpi in kstest[day]:
                        out[day][kpi].at[str(c[0]),str(c[1])] = r[day][kpi][0]
                        out[day][kpi].at[str(c[1]),str(c[0])] = r[day][kpi][0]
    return(out)

def buildKSTESTMatrixFromMPTEST(result):
    KS = pd.DataFrame()
    for res in result:
        cell1 = res.index.values[0]
        cell2 = res.columns.values[0]
        pvalue = res.loc[res.index.values[0]][res.columns.values[0]] 
        KS.at[str(cell1),str(cell2)] = pvalue
        KS.at[str(cell2),str(cell1)] = pvalue
    KS = KS.fillna(1)
    return(KS)

def Statistics(ZoneDataFrame,cell1,kpiList,dayList, wh = True):
    auxList = [kpi for kpi in kpiList]
    auxList.append('DAY_')
    cell1_kpi = ZoneDataFrame.loc[ZoneDataFrame['CellName']==cell1][auxList]
    cell1_kpi['DAY_'] = pd.to_datetime(cell1_kpi['DAY_'],infer_datetime_format=True)
    cell1_kpi = cell1_kpi.set_index('DAY_')
    out = pd.DataFrame()
    if wh:
        cell1_kpi = cell1_kpi.between_time('7:00','23:00')
        cell1_kpi['DAY_'] = cell1_kpi.index
        cell1_kpi['DAY_'] = pd.to_datetime(cell1_kpi['DAY_'],infer_datetime_format=True)
    for day in dayList:
        mask1 = (cell1_kpi['DAY_'] >=day) & (cell1_kpi['DAY_'] <day+timedelta(days=1) ) 
        for kpi in kpiList:
            out.at[str(day.date()),kpi+' mean'] = cell1_kpi.loc[mask1][kpi].mean()
            out.at[str(day.date()),kpi+' min'] = cell1_kpi.loc[mask1][kpi].min()
            out.at[str(day.date()),kpi+' max'] = cell1_kpi.loc[mask1][kpi].max()
               
    return(out)

print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)
print('Matplotlib version ' + matplotlib.__version__)

Python version 3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
Pandas version 0.23.0
Matplotlib version 2.2.2


In [3]:
def get_config(local_fic_conf, local_type):
    
    # print(local_type)
    liste_retour = []
    fic_config = open(local_fic_conf, "r")
    fin = 0 
    
    for line in fic_config:
        # print(line)
        line = line.replace("\n", "")        
        if line.find(local_type)>=0 and fin == 0 :
            # print("trouve")
            for line in fic_config :
                if line != "\n" and fin==0:
                    line = line.replace("\n", "")
                    liste_retour.append(line)
                else:
                    # print("fin trouve")
                    fin = 1
    fic_config.close()
    return(liste_retour)

def extract_list_from_file(local_file):
    loc_file = open(local_file, "r")
    data_read=[]
    for line in loc_file:
        line = line.replace("\n", "")
        data_read.append(line)
    loc_file.close()
    
    return (data_read)



###################
# global variables
###################
local_path = 'C:/Users/euripab/Desktop/ML Extraction/'
config_file = "config_python_max_jour.txt"

config_group = get_config(local_path + "/" + config_file, "config_group")

list_group=[]
inventaire_groupe=[]
inventaire_selected_struct=[]
if len(config_group)==1 :
    # extract group configuration
    list_group = extract_list_from_file(local_path + "/" + config_group[0])
    groupe_definition = {}
    for groupe_def in list_group :
        groupe_def2 = groupe_def.split(";")
        inventaire_selected_struct.append(groupe_def2[1])
        if not groupe_def2[0] in inventaire_groupe :
            inventaire_groupe.append(groupe_def2[0])
        groupe_definition[groupe_def2[1]] = groupe_def2[0]
        
# pour extraire les donnees de chaque groupe
# passer de liste a dataframe ()
array_list_group=[]
# pb : on garde les entetes dans les donnees -> a corriger
# les colonnes du dataframe ne sontpas nommees
for item in list_group:
    split_item = item.split(";")
    array_list_group.append(split_item)

df_list_group = pd.DataFrame(array_list_group)

# extraction des donnees du groupe residentiel
# df_list_group.loc[df_list_group[0]=='residentiel'][1]
# ResCellsKpi = alldata_kpis.loc[alldata_kpis['CellName'].isin(df_list_group.loc[df_list_group[0]=='residentiel'][1])]

# enlever 'TDC4GData', absent des donnees sources dans certains fichiers extraits de Ipod LTE
# kpiList = ['CQImoyen','TDC4GData','TrafficDataDL','DemandesVoix4G',
#            'VoIPqualitymeasinsuffstat','NombredeConnexionsPS','nbUEVoIPqualityOK',
#            'Nbrtentatives_QCi5__TEAVoix','TraficDataUL']
    
kpiList = ['CQImoyen','TraficDataDL','TraficDataUL',
           'VoIPqualitymeasinsuffstat','CROZON_NombredeConnexionsPS','nbUEVoIPqualityOK',
           'Nbrtentatives_QCi5__TEAVoix']
        

In [ ]:
#P Value Average
computing_type = "single"
liste_data = os.listdir(local_path + "/data/")
time_struct={}
last_group = ''
for current_data in liste_data:
    current_group = current_data.replace("_D.xlsx", "")
    current_group = current_group.replace("_L.xlsx", "")
    current_group = current_group.replace("_T.xlsx", "")
    current_group = current_group.replace("_P.xlsx", "")
    current_group = current_group.replace(".xlsx", "")
    if current_group != last_group:
        time_struct={}
    last_group = current_group    
    ZoneCellsKpi=pd.read_excel(local_path + "/data/" + current_data)
    fic_result= local_path + "/KSTest/results1_Pvalues/" + "KSTest_" + current_data 
    fic_time= local_path + "/KSTest/compute_time/" + "time_KS_" + current_group + ".xlsx"
    writer_results = pd.ExcelWriter(fic_result)
    writer_time = pd.ExcelWriter(fic_time)
    if computing_type == 'single':
        time_struct[current_data]={}
        print("Compute KS_Single fichier : {}, groupe : {})".format(current_data, current_group))
        for kpi in kpiList:
            time_struct[current_data][kpi]={}
            time_struct[current_data][kpi]["delta_t"]=[]
            start_time = time.time()
            KpiPvalueMatrix = KSTestAllCellsPerDayAverage(ZoneCellsKpi,kpi)
            end_time = time.time()
            time_struct[current_data][kpi]["delta_t"].append(end_time-start_time)                
            KpiPvalueMatrix.to_excel(writer_results,sheet_name=kpi)
            df_time_struct = pd.DataFrame(time_struct)
            df_time_struct.to_excel(writer_time,sheet_name=current_group)
        writer_time.save()
        writer_results.save()

    elif computing_type == 'parallel':
        time_struct[current_data]={}
        print("Compute KS_parallel fichier : {}".format(current_data))
        for kpi in kpiList:
            time_struct[current_data][kpi]={}
            time_struct[current_data][kpi]["delta_t"]=[]
            start_time = time.time()
            results = KSTestAllCellsPerDayAverageParallel(ZoneCellsKpi,kpi)
            end_time = time.time()  
            KpiPvalueMatrix = buildKSTESTMatrixFromMPTEST(results)
            del results
            time_struct[current_data][kpi]["delta_t"].append(end_time-start_time)                 
            KpiPvalueMatrix.to_excel(writer_results,sheet_name=kpi)
            df_time_struct = pd.DataFrame(time_struct)
            df_time_struct.to_excel(writer_time,sheet_name=current_group)
        writer_time.save()
        writer_results.save()

In [ ]:
#P value per Day
computing_type = "parallel"
liste_data = os.listdir(local_path + "/data/")
time_struct={}
last_group = ''
dayList = np.array([datetime(2018,8,7)+timedelta(days=i) for i in range(0,32)])
for current_data in liste_data:
    current_group = current_data.replace("_D.xlsx", "")
    current_group = current_group.replace("_L.xlsx", "")
    current_group = current_group.replace("_T.xlsx", "")
    current_group = current_group.replace("_P.xlsx", "")
    current_group = current_group.replace(".xlsx", "")
    if current_group != last_group:
        time_struct={}
    last_group = current_group    
    ZoneCellsKpi=pd.read_excel(local_path + "/data/" + current_data)
    if computing_type == 'single':
        time_struct[current_data]={}
        print("Compute KS_Single fichier : {}, groupe : {})".format(current_data, current_group))
        results = KSTestAllCells(ZoneCellsKpi,kpiList,dayList)
        for day in dayList:
            fic_result= local_path + "/KSTest/results3_PvaluesDays/" + "KSTest_"+str(day.date())+'_' + current_data 
            fic_time= local_path + "/KSTest/compute_time/" + "time_KS_" +str(day.date())+'_' +current_group + ".xlsx"
            writer_results = pd.ExcelWriter(fic_result)
            writer_time = pd.ExcelWriter(fic_time)
            for kpi in kpiList:
                KpiPvalueMatrix = results[day][kpi]
                KpiPvalueMatrix.to_excel(writer_results,sheet_name=kpi)
        writer_time.save()
        writer_results.save()

    elif computing_type == 'parallel':
        time_struct[current_data]={}
        print("Compute KS_Single fichier : {}, groupe : {})".format(current_data, current_group))
        results = KSTestAllCellsParallel(ZoneCellsKpi,kpiList,dayList)
        for day in dayList:
            fic_result= local_path + "/KSTest/results3_PvaluesDays/" + "KSTest_"+str(day.date())+'_' + current_data 
            fic_time= local_path + "/KSTest/compute_time/" + "time_KS_" +str(day.date())+'_' +current_group + ".xlsx"
            writer_results = pd.ExcelWriter(fic_result)
            writer_time = pd.ExcelWriter(fic_time)
            for kpi in kpiList:
                KpiPvalueMatrix = results[day][kpi]
                KpiPvalueMatrix.to_excel(writer_results,sheet_name=kpi)
        writer_time.save()
        writer_results.save()

Compute KS_Single fichier : all.xlsx, groupe : all)
0.0


In [21]:
#Clustering
liste_data = os.listdir(local_path + "/KSTest/results3_PvaluesDays/")
time_struct={}
last_group = ''
for current_data in liste_data:
    current_group = current_data.replace("_D.xlsx", "")
    current_group = current_group.replace("_L.xlsx", "")
    current_group = current_group.replace("_T.xlsx", "")
    current_group = current_group.replace("_P.xlsx", "")
    current_group = current_group.replace(".xlsx", "")
    if current_group != last_group:
        time_struct={}
    last_group = current_group    

    fic_result= local_path + "/KSTest/results4_ClustersDays/" + "clustering_" + current_data
    fic_time= local_path + "/KSTest/compute_time/" + "time_clustering_" + current_group + ".xlsx"
    writer_results = pd.ExcelWriter(fic_result)
    writer_time = pd.ExcelWriter(fic_time)
    print("Compute Clustering fichier : {}, groupe : {})".format(current_data, current_group[:30]))
    path = local_path + "/KSTest/results3_PvaluesDays/" +current_data
    output = generateKpiLabeledDataFromExcelKSTEST(path, kpiList)
    time_struct[current_data]=output[1]
    output[0].to_excel(writer_results,sheet_name=current_group[:30])
    df_time_struct = pd.DataFrame(time_struct)
    df_time_struct.to_excel(writer_time,sheet_name=current_group[:30])
    writer_time.save()
    writer_results.save()

Compute Clustering fichier : KSTest_2018-08-07_all.xlsx, groupe : KSTest_2018-08-07_all)
Compute Clustering fichier : KSTest_2018-08-07_Chatelet_les_halles.xlsx, groupe : KSTest_2018-08-07_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-07_Chatelet_les_halles_D.xlsx, groupe : KSTest_2018-08-07_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-07_Chatelet_les_halles_L.xlsx, groupe : KSTest_2018-08-07_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-07_Chatelet_les_halles_P.xlsx, groupe : KSTest_2018-08-07_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-07_Chatelet_les_halles_T.xlsx, groupe : KSTest_2018-08-07_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-07_concorde.xlsx, groupe : KSTest_2018-08-07_concorde)
Compute Clustering fichier : KSTest_2018-08-07_concorde_D.xlsx, groupe : KSTest_2018-08-07_concorde)
Compute Clustering fichier : KSTest_2018-08-07_concorde_L.xlsx, groupe : KSTest_2018-08-07_concorde)
Compute Clustering fichier : KST

Compute Clustering fichier : KSTest_2018-08-09_rive_droite_seine_P.xlsx, groupe : KSTest_2018-08-09_rive_droite_)
Compute Clustering fichier : KSTest_2018-08-09_rive_droite_seine_T.xlsx, groupe : KSTest_2018-08-09_rive_droite_)
Compute Clustering fichier : KSTest_2018-08-10_all.xlsx, groupe : KSTest_2018-08-10_all)
Compute Clustering fichier : KSTest_2018-08-10_Chatelet_les_halles.xlsx, groupe : KSTest_2018-08-10_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-10_Chatelet_les_halles_D.xlsx, groupe : KSTest_2018-08-10_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-10_Chatelet_les_halles_L.xlsx, groupe : KSTest_2018-08-10_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-10_Chatelet_les_halles_P.xlsx, groupe : KSTest_2018-08-10_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-10_Chatelet_les_halles_T.xlsx, groupe : KSTest_2018-08-10_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-10_concorde.xlsx, groupe : KSTest_2018-08-10_concorde)
Comput

Compute Clustering fichier : KSTest_2018-08-12_rive_droite_seine_D.xlsx, groupe : KSTest_2018-08-12_rive_droite_)
Compute Clustering fichier : KSTest_2018-08-12_rive_droite_seine_L.xlsx, groupe : KSTest_2018-08-12_rive_droite_)
Compute Clustering fichier : KSTest_2018-08-12_rive_droite_seine_P.xlsx, groupe : KSTest_2018-08-12_rive_droite_)
Compute Clustering fichier : KSTest_2018-08-12_rive_droite_seine_T.xlsx, groupe : KSTest_2018-08-12_rive_droite_)
Compute Clustering fichier : KSTest_2018-08-13_all.xlsx, groupe : KSTest_2018-08-13_all)
Compute Clustering fichier : KSTest_2018-08-13_Chatelet_les_halles.xlsx, groupe : KSTest_2018-08-13_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-13_Chatelet_les_halles_D.xlsx, groupe : KSTest_2018-08-13_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-13_Chatelet_les_halles_L.xlsx, groupe : KSTest_2018-08-13_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-13_Chatelet_les_halles_P.xlsx, groupe : KSTest_2018-08-13_Chatele

Compute Clustering fichier : KSTest_2018-08-15_rive_droite_seine.xlsx, groupe : KSTest_2018-08-15_rive_droite_)
Compute Clustering fichier : KSTest_2018-08-15_rive_droite_seine_D.xlsx, groupe : KSTest_2018-08-15_rive_droite_)
Compute Clustering fichier : KSTest_2018-08-15_rive_droite_seine_L.xlsx, groupe : KSTest_2018-08-15_rive_droite_)
Compute Clustering fichier : KSTest_2018-08-15_rive_droite_seine_P.xlsx, groupe : KSTest_2018-08-15_rive_droite_)
Compute Clustering fichier : KSTest_2018-08-15_rive_droite_seine_T.xlsx, groupe : KSTest_2018-08-15_rive_droite_)
Compute Clustering fichier : KSTest_2018-08-16_all.xlsx, groupe : KSTest_2018-08-16_all)
Compute Clustering fichier : KSTest_2018-08-16_Chatelet_les_halles.xlsx, groupe : KSTest_2018-08-16_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-16_Chatelet_les_halles_D.xlsx, groupe : KSTest_2018-08-16_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-16_Chatelet_les_halles_L.xlsx, groupe : KSTest_2018-08-16_Chatelet_le

Compute Clustering fichier : KSTest_2018-08-18_residentiel_P.xlsx, groupe : KSTest_2018-08-18_residentiel)
Compute Clustering fichier : KSTest_2018-08-18_residentiel_T.xlsx, groupe : KSTest_2018-08-18_residentiel)
Compute Clustering fichier : KSTest_2018-08-18_rive_droite_seine.xlsx, groupe : KSTest_2018-08-18_rive_droite_)
Compute Clustering fichier : KSTest_2018-08-18_rive_droite_seine_D.xlsx, groupe : KSTest_2018-08-18_rive_droite_)
Compute Clustering fichier : KSTest_2018-08-18_rive_droite_seine_L.xlsx, groupe : KSTest_2018-08-18_rive_droite_)
Compute Clustering fichier : KSTest_2018-08-18_rive_droite_seine_P.xlsx, groupe : KSTest_2018-08-18_rive_droite_)
Compute Clustering fichier : KSTest_2018-08-18_rive_droite_seine_T.xlsx, groupe : KSTest_2018-08-18_rive_droite_)
Compute Clustering fichier : KSTest_2018-08-19_all.xlsx, groupe : KSTest_2018-08-19_all)
Compute Clustering fichier : KSTest_2018-08-19_Chatelet_les_halles.xlsx, groupe : KSTest_2018-08-19_Chatelet_les)
Compute Cluster

...

Compute Clustering fichier : KSTest_2018-08-24_residentiel.xlsx, groupe : KSTest_2018-08-24_residentiel)
Compute Clustering fichier : KSTest_2018-08-24_residentiel_D.xlsx, groupe : KSTest_2018-08-24_residentiel)
Compute Clustering fichier : KSTest_2018-08-24_residentiel_L.xlsx, groupe : KSTest_2018-08-24_residentiel)
Compute Clustering fichier : KSTest_2018-08-24_residentiel_P.xlsx, groupe : KSTest_2018-08-24_residentiel)
Compute Clustering fichier : KSTest_2018-08-24_residentiel_T.xlsx, groupe : KSTest_2018-08-24_residentiel)
Compute Clustering fichier : KSTest_2018-08-24_rive_droite_seine.xlsx, groupe : KSTest_2018-08-24_rive_droite_)
Compute Clustering fichier : KSTest_2018-08-24_rive_droite_seine_D.xlsx, groupe : KSTest_2018-08-24_rive_droite_)
Compute Clustering fichier : KSTest_2018-08-24_rive_droite_seine_L.xlsx, groupe : KSTest_2018-08-24_rive_droite_)
Compute Clustering fichier : KSTest_2018-08-24_rive_droite_seine_P.xlsx, groupe : KSTest_2018-08-24_rive_droite_)
Compute Clust

Compute Clustering fichier : KSTest_2018-08-27_Gare_St_lazare_P.xlsx, groupe : KSTest_2018-08-27_Gare_St_laza)
Compute Clustering fichier : KSTest_2018-08-27_Gare_St_lazare_T.xlsx, groupe : KSTest_2018-08-27_Gare_St_laza)
Compute Clustering fichier : KSTest_2018-08-27_residentiel.xlsx, groupe : KSTest_2018-08-27_residentiel)
Compute Clustering fichier : KSTest_2018-08-27_residentiel_D.xlsx, groupe : KSTest_2018-08-27_residentiel)
Compute Clustering fichier : KSTest_2018-08-27_residentiel_L.xlsx, groupe : KSTest_2018-08-27_residentiel)
Compute Clustering fichier : KSTest_2018-08-27_residentiel_P.xlsx, groupe : KSTest_2018-08-27_residentiel)
Compute Clustering fichier : KSTest_2018-08-27_residentiel_T.xlsx, groupe : KSTest_2018-08-27_residentiel)
Compute Clustering fichier : KSTest_2018-08-27_rive_droite_seine.xlsx, groupe : KSTest_2018-08-27_rive_droite_)
Compute Clustering fichier : KSTest_2018-08-27_rive_droite_seine_D.xlsx, groupe : KSTest_2018-08-27_rive_droite_)
Compute Clustering 

Compute Clustering fichier : KSTest_2018-08-30_Gare_St_lazare_D.xlsx, groupe : KSTest_2018-08-30_Gare_St_laza)
Compute Clustering fichier : KSTest_2018-08-30_Gare_St_lazare_L.xlsx, groupe : KSTest_2018-08-30_Gare_St_laza)
Compute Clustering fichier : KSTest_2018-08-30_Gare_St_lazare_P.xlsx, groupe : KSTest_2018-08-30_Gare_St_laza)
Compute Clustering fichier : KSTest_2018-08-30_Gare_St_lazare_T.xlsx, groupe : KSTest_2018-08-30_Gare_St_laza)
Compute Clustering fichier : KSTest_2018-08-30_residentiel.xlsx, groupe : KSTest_2018-08-30_residentiel)
Compute Clustering fichier : KSTest_2018-08-30_residentiel_D.xlsx, groupe : KSTest_2018-08-30_residentiel)
Compute Clustering fichier : KSTest_2018-08-30_residentiel_L.xlsx, groupe : KSTest_2018-08-30_residentiel)
Compute Clustering fichier : KSTest_2018-08-30_residentiel_P.xlsx, groupe : KSTest_2018-08-30_residentiel)
Compute Clustering fichier : KSTest_2018-08-30_residentiel_T.xlsx, groupe : KSTest_2018-08-30_residentiel)
Compute Clustering fich

Compute Clustering fichier : KSTest_2018-09-02_Gare_St_lazare.xlsx, groupe : KSTest_2018-09-02_Gare_St_laza)
Compute Clustering fichier : KSTest_2018-09-02_Gare_St_lazare_D.xlsx, groupe : KSTest_2018-09-02_Gare_St_laza)
Compute Clustering fichier : KSTest_2018-09-02_Gare_St_lazare_L.xlsx, groupe : KSTest_2018-09-02_Gare_St_laza)
Compute Clustering fichier : KSTest_2018-09-02_Gare_St_lazare_P.xlsx, groupe : KSTest_2018-09-02_Gare_St_laza)
Compute Clustering fichier : KSTest_2018-09-02_Gare_St_lazare_T.xlsx, groupe : KSTest_2018-09-02_Gare_St_laza)
Compute Clustering fichier : KSTest_2018-09-02_residentiel.xlsx, groupe : KSTest_2018-09-02_residentiel)
Compute Clustering fichier : KSTest_2018-09-02_residentiel_D.xlsx, groupe : KSTest_2018-09-02_residentiel)
Compute Clustering fichier : KSTest_2018-09-02_residentiel_L.xlsx, groupe : KSTest_2018-09-02_residentiel)
Compute Clustering fichier : KSTest_2018-09-02_residentiel_P.xlsx, groupe : KSTest_2018-09-02_residentiel)
Compute Clustering fi

Compute Clustering fichier : KSTest_2018-09-05_concorde_P.xlsx, groupe : KSTest_2018-09-05_concorde)
Compute Clustering fichier : KSTest_2018-09-05_concorde_T.xlsx, groupe : KSTest_2018-09-05_concorde)
Compute Clustering fichier : KSTest_2018-09-05_Gare_St_lazare.xlsx, groupe : KSTest_2018-09-05_Gare_St_laza)
Compute Clustering fichier : KSTest_2018-09-05_Gare_St_lazare_D.xlsx, groupe : KSTest_2018-09-05_Gare_St_laza)
Compute Clustering fichier : KSTest_2018-09-05_Gare_St_lazare_L.xlsx, groupe : KSTest_2018-09-05_Gare_St_laza)
Compute Clustering fichier : KSTest_2018-09-05_Gare_St_lazare_P.xlsx, groupe : KSTest_2018-09-05_Gare_St_laza)
Compute Clustering fichier : KSTest_2018-09-05_Gare_St_lazare_T.xlsx, groupe : KSTest_2018-09-05_Gare_St_laza)
Compute Clustering fichier : KSTest_2018-09-05_residentiel.xlsx, groupe : KSTest_2018-09-05_residentiel)
Compute Clustering fichier : KSTest_2018-09-05_residentiel_D.xlsx, groupe : KSTest_2018-09-05_residentiel)
Compute Clustering fichier : KSTe

In [22]:
#Notmal Statistics
kpiList = ['CQImoyen','TraficDataDL','TraficDataUL',
           'VoIPqualitymeasinsuffstat','CROZON_NombredeConnexionsPS','nbUEVoIPqualityOK',
           'Nbrtentatives_QCi5__TEAVoix']
computing_type = "single"
liste_data = os.listdir(local_path + "/data/")
time_struct={}
last_group = ''
dayList = np.array([datetime(2018,8,7)+timedelta(days=i) for i in range(0,32)])
for current_data in liste_data:
    current_group = current_data.replace("_D.xlsx", "")
    current_group = current_group.replace("_L.xlsx", "")
    current_group = current_group.replace("_T.xlsx", "")
    current_group = current_group.replace("_P.xlsx", "")
    current_group = current_group.replace(".xlsx", "")
    if current_group != last_group:
        time_struct={}
    last_group = current_group    
    ZoneDataFrame=pd.read_excel(local_path + "/data/" + current_data)
    if computing_type == 'single':
        time_struct[current_data]={}
        print("Compute Statistics_Single fichier : {}, groupe : {})".format(current_data, current_group))
        for cell in ZoneDataFrame['CellName'].unique():
            results = Statistics(ZoneCellsKpi,cell,kpiList,dayList)
            fic_result= local_path + "/Statistics/results1_MeanMaxMin_PerDay/" +'MeanMaxMin_PerDay_'+current_data.replace(".xlsx", "")+'_' +str(cell)+'.xlsx'
            writer_results = pd.ExcelWriter(fic_result)
            results.to_excel(writer_results)
            writer_results.save()

    elif computing_type == 'parallel':
        time_struct[current_data]={}
        print("Compute KS_Single fichier : {}, groupe : {})".format(current_data, current_group))
        results = KSTestAllCellsParallel(ZoneCellsKpi,kpiList,dayList)
        for day in dayList:
            fic_result= local_path + "/KSTest/results5_ClusteringTimeSeries/" +'clustering_ts_'+groupe.replace(".xlsx", "")+'_' +str(cell)+'.xlsx'
            fic_time= local_path + "/KSTest/compute_time/" + "time_KS_" +str(day.date())+'_' +current_group + ".xlsx"
            writer_results = pd.ExcelWriter(fic_result)
            writer_time = pd.ExcelWriter(fic_time)
            for kpi in kpiList:
                KpiPvalueMatrix = results[day][kpi]
                KpiPvalueMatrix.to_excel(writer_results,sheet_name=kpi)
        writer_time.save()
        writer_results.save()


Compute Statistics_Single fichier : all.xlsx, groupe : all)
Compute Statistics_Single fichier : Chatelet_les_halles.xlsx, groupe : Chatelet_les_halles)
Compute Statistics_Single fichier : Chatelet_les_halles_D.xlsx, groupe : Chatelet_les_halles)
Compute Statistics_Single fichier : Chatelet_les_halles_L.xlsx, groupe : Chatelet_les_halles)
Compute Statistics_Single fichier : Chatelet_les_halles_P.xlsx, groupe : Chatelet_les_halles)
Compute Statistics_Single fichier : Chatelet_les_halles_T.xlsx, groupe : Chatelet_les_halles)
Compute Statistics_Single fichier : concorde.xlsx, groupe : concorde)
Compute Statistics_Single fichier : concorde_D.xlsx, groupe : concorde)
Compute Statistics_Single fichier : concorde_L.xlsx, groupe : concorde)
Compute Statistics_Single fichier : concorde_P.xlsx, groupe : concorde)
Compute Statistics_Single fichier : concorde_T.xlsx, groupe : concorde)
Compute Statistics_Single fichier : Gare_St_lazare.xlsx, groupe : Gare_St_lazare)
Compute Statistics_Single fichie

In [23]:
def linearMap(x,r,MAX,MIN):
    return((1-r)*(x-MAX)/(MAX-MIN)+1)
    

In [10]:
kpiList = ['CQImoyen','TraficDataDL','TraficDataUL',
           'VoIPqualitymeasinsuffstat','CROZON_NombredeConnexionsPS','nbUEVoIPqualityOK',
           'Nbrtentatives_QCi5__TEAVoix']
#Creating Clustering Time Series Data
computing_type = "single"
liste_data = os.listdir(local_path + "/KSTest/results4_ClustersDays/")
time_struct={}
last_group = ''
auxList = []
num = 0
groupe = 'all.xlsx'
for current_data in liste_data:
    if current_data.endswith(groupe): 
        ZoneCellsKpi = pd.read_excel(local_path + "/KSTest/results4_ClustersDays/" + current_data)
        for kpi in ZoneCellsKpi.columns:
            #ZoneCellsKpi[kpi] =len(ZoneCellsKpi)/ZoneCellsKpi.groupby(kpi)[kpi].transform('count')
            ZoneCellsKpi[kpi] = ZoneCellsKpi.groupby(kpi)[kpi].transform('count')/len(ZoneCellsKpi)
            ZoneCellsKpi[kpi] = (ZoneCellsKpi[kpi]-ZoneCellsKpi[kpi].min())/(ZoneCellsKpi[kpi].max()-ZoneCellsKpi[kpi].min())
            #ZoneCellsKpi[kpi] = ZoneCellsKpi[kpi]*ZoneCellsKpi[kpi].apply(lambda x: linearMap(x,0.1,ZoneCellsKpi[kpi].max(),ZoneCellsKpi[kpi].min()))
        ZoneCellsKpi.index.name = 'Cell_Name'
        ZoneCellsKpi['Tec'] = [cell[0] for cell in ZoneCellsKpi.index]
        ZoneCellsKpi.columns = [str(col)+' '+str(num) for col in ZoneCellsKpi.columns]
        num = num+1
        auxList.append(ZoneCellsKpi)
alldaysClusters = functools.reduce(lambda left,right: pd.merge(left,right,on='Cell_Name'), auxList)
kpiList.append('Tec')
for cell in alldaysClusters.index:
    cellAllKpi = pd.DataFrame()
    for kpi in kpiList:
        cellkpiSeries = alldaysClusters.loc[cell][alldaysClusters.columns[alldaysClusters.columns.str.startswith(kpi)]]
        cellAllKpi[kpi] = np.array(cellkpiSeries)
    fic_result= local_path + "/KSTest/results5_ClusteringTimeSeries/" +'clustering_ts_'+groupe.replace(".xlsx", "")+'_' +str(cell)+'.xlsx'
    writer_results = pd.ExcelWriter(fic_result)
    cellAllKpi.to_excel(writer_results)
    writer_results.save()

    

In [11]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [(name+'(t-%d)' % (i)) for name in data.columns]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [name+'(t)' for name in data.columns]
		else:
			names += [(name+'(t+%d)' % (i)) for name in data.columns]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg


In [13]:
groupe= 'all'
#Creating Clustering Supervised Data
computing_type = "single"
liste_data = os.listdir(local_path + "/KSTest/results5_ClusteringTimeSeries/")
allCellSupervisedData = pd.DataFrame()
predictive_time_units = 1
to_predict_time_units = 7
for current_data in liste_data:
    if groupe in current_data:
        timeSeries = pd.read_excel(local_path + "/KSTest/results5_ClusteringTimeSeries/" + current_data)
        cellSupervisedData = series_to_supervised(timeSeries, n_in=predictive_time_units, n_out=to_predict_time_units, dropnan=True)
        allCellSupervisedData = allCellSupervisedData.append(cellSupervisedData, ignore_index=True)
    
fic_result= local_path + "/KSTest/results6_ClusteringSupervisedData/" +'supervised_' +groupe+str(predictive_time_units)+'_'+str(to_predict_time_units)+'.xlsx'
writer_results = pd.ExcelWriter(fic_result)
allCellSupervisedData.to_excel(writer_results)
writer_results.save()

In [ ]:
cellKpi= pd.read_excel(r'C:\Users\euripab\Desktop\ML Extraction\KSTEST\results5_ClusteringTimeSeries\clustering_ts_all_D10562A.xlsx')
cellKpi.plot(y =cellAllKpi.columns, kind='line',figsize = (15,10))

In [9]:
ZoneCellsKpi=pd.read_excel(r'C:\Users\euripab\Desktop\ML Extraction\data\all.xlsx')

In [10]:
dayList = np.array([datetime(2018,8,7)+timedelta(days=i) for i in range(0,32)])

In [11]:
results = KSTestAllCells(ZoneCellsKpi,kpiList,dayList)

In [20]:
for day in dayList:
    fic_result= local_path + "/KSTest/results3_PvaluesDays/" + "KSTest_"+str(day.date())+'_all.xlsx' 
    writer_results = pd.ExcelWriter(fic_result)
    for kpi in kpiList:
        KpiPvalueMatrix = results[day][kpi]
        KpiPvalueMatrix.to_excel(writer_results,sheet_name=kpi)
    writer_results.save()

In [23]:
groupe= 'all'
#Creating Clustering Supervised Data
computing_type = "single"
liste_data = os.listdir(local_path + "/KSTest/results5_ClusteringTimeSeries/")
allCellSupervisedData = pd.DataFrame()
predictive_time_units = 1
to_predict_time_units = 7
for current_data in liste_data:
    if groupe in current_data:
        timeSeries1 = pd.read_excel(local_path + "/KSTest/results5_ClusteringTimeSeries/" + current_data)
        cellSupervisedData = series_to_supervised(timeSeries1, n_in=predictive_time_units, n_out=to_predict_time_units, dropnan=True)
        timeSeries1 = pd.read_excel(local_path + "/Statistics/results5_ClusteringTimeSeries/" + current_data)
        cellSupervisedData = series_to_supervised(timeSeries1, n_in=predictive_time_units, n_out=to_predict_time_units, dropnan=True)
        allCellSupervisedData = allCellSupervisedData.append(cellSupervisedData, ignore_index=True)
    
fic_result= local_path + "/KSTest/results6_ClusteringSupervisedData/" +'supervised_' +groupe+str(predictive_time_units)+'_'+str(to_predict_time_units)+'.xlsx'
writer_results = pd.ExcelWriter(fic_result)
allCellSupervisedData.to_excel(writer_results)
writer_results.save()

,Site 0,Tec 0,Azimuth 0,CQImoyen Label 0,TraficDataDL Label 0,TraficDataUL Label 0,VoIPqualitymeasinsuffstat Label 0,CROZON_NombredeConnexionsPS Label 0,nbUEVoIPqualityOK Label 0,Nbrtentatives_QCi5__TEAVoix Label 0,...,Site 31,Tec 31,Azimuth 31,CQImoyen Label 31,TraficDataDL Label 31,TraficDataUL Label 31,VoIPqualitymeasinsuffstat Label 31,CROZON_NombredeConnexionsPS Label 31,nbUEVoIPqualityOK Label 31,Nbrtentatives_QCi5__TEAVoix Label 31
Cell_Name,,,,,,,,,,,,,,,,,,,,,
L10939C,1.000,L,0.000,0.166667,0.909091,0.269231,0.552632,0.176471,0.256410,0.000000,...,1.000,L,0.000,0.266667,0.478261,0.136364,0.68750,0.052632,0.275,0.00
L11183A,1.000,L,1.000,0.333333,0.272727,0.230769,0.315789,0.176471,0.948718,0.529412,...,1.000,L,1.000,0.333333,0.913043,1.000000,0.87500,0.631579,1.000,0.70
L11183B,1.000,L,0.625,0.555556,0.090909,0.923077,0.026316,0.352941,1.000000,1.000000,...,1.000,L,0.625,1.000000,0.217391,0.045455,0.96875,0.421053,0.700,0.35
L11183C,1.000,L,0.000,0.277778,0.590909,0.461538,0.078947,0.411765,0.820513,0.588235,...,1.000,L,0.000,0.600000,0.695652,0.454545,0.03125,0.157895,1.000,0.40
L14837A,0.375,L,1.000,0.277778,1.000000,1.000000,0.842105,0.294118,0.820513,1.000000,...,0.375,L,1.000,0.200000,0.565217,0.681818,0.31250,0.263158,1.000,0.15
L14837B,0.375,L,0.625,0.333333,1.000000,1.000000,0.842105,0.764706,0.230769,1.000000,...,0.375,L,0.625,0.400000,0.913043,1.000000,0.81250,0.315789,0.200,1.00
L15229A,1.000,L,1.000,0.388889,1.000000,0.923077,0.868421,0.352941,1.000000,1.000000,...,1.000,L,1.000,0.200000,0.826087,0.863636,0.96875,0.052632,0.700,0.35
L15229B,1.000,L,0.625,0.388889,1.000000,0.923077,0.263158,0.352941,1.000000,0.823529,...,1.000,L,0.625,0.400000,0.913043,1.000000,0.37500,0.315789,0.600,1.00
L15229C,1.000,L,0.000,0.388889,1.000000,0.153846,1.000000,0.764706,1.000000,0.411765,...,1.000,L,0.000,0.666667,0.956522,0.636364,1.00000,0.368421,0.175,0.35
